In [2]:
PROJECT = !gcloud config get-value project # returns SList
PROJECT = PROJECT[0] # gets first element in list -> str
REGION = "us-central1"  
import os
os.environ["PROJECT"] = PROJECT
os.environ["REGION"] = REGION

In [3]:
from google.cloud import bigquery
bq = bigquery.Client(project=PROJECT)

# Preprocessing

1. for each table in the dataset, change location link to our storage

`gs://gcs-public-data--labeled-patents/us_006.pdf` --> `gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf/us_006.pdf`

2. create new col "filename"

Get table

In [4]:
query_string = """
#standardSQL
SELECT
    *
FROM
    `qwiklabs-gcp-00-373ac55d0e0a.labeled_patents.extracted_data`
"""

extracted_data = bq.query(query_string).to_dataframe()

Copy into new df variable

In [5]:
extracted_data_modified = extracted_data.copy(deep=True)

In [6]:
original = "gs://gcs-public-data--labeled-patents/us_006.pdf"

In [7]:
# os.path.basename(original)

In [8]:
# IMPORTANT: NOT WORKING -- TODO
# import re
# pattern = r"gs:\/\/(.*?)/.*\.pdf"
# re.search(pattern, original).group(1)

In [9]:
def change_gcs_path(original):
    split_path = os.path.split(original)
    new_path = os.path.join("gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/pdf" ,split_path[1])
    return new_path

def get_filename(original):
    return os.path.basename(original)

In [10]:
extracted_data_modified["gcs_path"] = extracted_data_modified["gcs_path"].apply(change_gcs_path)

In [11]:
extracted_data_modified["filename"] = extracted_data_modified["gcs_path"].apply(get_filename)

In [12]:
extracted_data_modified

,gcs_path,issuer,language,publication_date,class_international,class_us,application_number,filing_date,priority_date_eu,representative_line_1_eu,applicant_line_1,inventor_line_1,title_line_1,number,filename
0,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,29.08.018,E04H 6/12,None,18157874.1,21.02.2018,22.02.2017,Liedtke & Partner Patentanw√§lte,SHB Hebezeugbau GmbH,"VOLGER, Alexander",STEUERUNGSSYSTEM F√úR AUTOMATISCHE PARKH√ÑUSER,EP 3 366 869 A1,espacenet_de2.pdf
1,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,A61B 1/00,None,18164602.7,28.03.2018,28.03.2017,"Gramm, Line, & Partner","Emmanouilidis, Nikos","Emmanouilidis, Nikos",MEDIZINISCHE VORRICHTUNG,EP 3 381 348 A1,espacenet_de78.pdf
2,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,H05B 6/12,None,18165514.3,03.04.2018,30.03.2017,None,BSH Hausger√§te GmbH,"Acero Acero, Jesus",VORRICHTUNG ZUR INDUKTIVEN ENERGIE√úBERTRAGUNG,EP 3 383 141 A2,espacenet_de73.pdf
3,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,H01Q 1/22,None,18164695.1,28.03.2018,31.03.2017,"Kruspig, Volkmar et al",Antennentechnik ABB Bad,"Harz, Thomas",ELEKTROTECHNISCHE EINRICHTUNG,EP 3 382 793 A1,espacenet_de77.pdf
4,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,DE,03.10.2018,A01K 31/00,None,18171005.4,05.02.2015,05.02.2014,Stork Bamberger Patentanw√§lte,Linco Food Systems A/S,"Thrane, Uffe",MASTH√ÑHNCHENCONTAINER ALS BESTANDTEIL EINER E...,EP 3 381 276 A1,espacenet_de56.pdf
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
338,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,30.08.2017,H04L 9/32,None,17157289.4,22.02.2017,25.02.2016,Brevalex,Commissariat √† l'Energie Atomique,"SAVRY, Olivier",M√âTHODE DE GESTION DE CERTIFICATS IMPLICITES ...,EP 3 211 826 A1,espacenet_fr25.pdf
339,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,30.09.2015,E21B 43/00,None,15305280.8,24.02.2015,25.03.2014,None,IFP Energies nouvelles,"Mesri, Youssef",PROC√âD√â DE CONSTRUCTION D'UN MAILLAGE OPTIMI...,EP 2 924 232 A2,espacenet_fr61.pdf
340,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,30.12.2015,H04L 29/06,None,15173447.2,23.06.2015,27.06.2014,"Delumeau, Fran√ßois Guy et al",Orange,"He, Ruan",PROCEDE DE GESTION DE CONTROLE D'ACCES DANS UN...,EP 2 961 130 A1,espacenet_fr45.pdf
341,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU,FR,31.08.2016,G06F 19/00,None,16156915.7,23.02.2016,25.02.2015,"Delumeau, Fran√ßois Guy et al",Biomnis,"BESSON, Michel L.",SYSTEME POUR L'ANALYSE D'ECHANTILLONS POUVANT ...,EP 3 062 251 A1,espacenet_fr100.pdf


In [13]:
# optional: Preprocess DE, FR files to substitue error characters

In [14]:
extracted_data_modified.shape

(343, 15)

In [15]:
extracted_data_modified.to_csv("./dataset/extracted_data.csv", index=False, header=False)

save modified table in BQ

In [16]:
!gsutil -m cp ./dataset/extracted_data.csv  gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/extracted_data.csv

Copying file://./dataset/extracted_data.csv [Content-Type=text/csv]...
/ [1/1 files][ 92.8 KiB/ 92.8 KiB] 100% Done                                    
Operation completed over 1 objects/92.8 KiB.                                     


In [17]:
# extracted_data_modified.shape

In [18]:
# TODO(developer): Set table_id to the ID of the table to create.
table_id = "qwiklabs-gcp-00-373ac55d0e0a.labeled_patents_preprocessed.extracted_data"

job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField("gcs_path", "STRING"),
        bigquery.SchemaField("issuer", "STRING"),
        bigquery.SchemaField("language", "STRING"),
        bigquery.SchemaField("publication_date", "STRING"),
        bigquery.SchemaField("class_international", "STRING"),
        bigquery.SchemaField("class_us", "STRING"),
        bigquery.SchemaField("application_number", "STRING"),
        bigquery.SchemaField("filing_date", "STRING"),
        bigquery.SchemaField("priority_date_eu", "STRING"),
        bigquery.SchemaField("representative_line_1_eu", "STRING"),
        bigquery.SchemaField("applicant_line_1", "STRING"),
        bigquery.SchemaField("inventor_line_1", "STRING"),
        bigquery.SchemaField("title_line_1", "STRING"),
        bigquery.SchemaField("number", "STRING"),
        bigquery.SchemaField("filename", "STRING"),
    ],
    skip_leading_rows=0,
    # The source format defaults to CSV, so the line below is optional.
    source_format=bigquery.SourceFormat.CSV,
)
uri = "gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/extracted_data.csv"

load_job = bq.load_table_from_uri(
    uri, table_id, job_config=job_config
)  # Make an API request.

load_job.result()  # Waits for the job to complete.

destination_table = bq.get_table(table_id)  # Make an API request.
print("Loaded {} rows.".format(destination_table.num_rows))

Loaded 343 rows.


repeat for the other tables as well

# Convert PDF to image

In [19]:
# General libraries:
import glob
import numpy as np

# Dealing with images:
import cv2
import matplotlib.pyplot as plt

# Google APIs:

# Specific PDF libraries:
#!conda install -c conda-forge poppler
!sudo apt-get install -y poppler-data
!sudo apt-get install -y poppler-utils
!pip install pdf2image
from pdf2image import convert_from_path

Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-data is already the newest version (0.4.9-2).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
poppler-utils is already the newest version (0.71.0-5).
0 upgraded, 0 newly installed, 0 to remove and 9 not upgraded.


In [32]:
image_dir = "./dataset/images"

if not os.path.exists(image_dir):
    os.mkdir(image_dir)

In [34]:
# file = os.listdir(pdf_dir)[0]
# os.path.join(image_dir, file[:-3] + "png")


'./dataset/images/us_067.png'

In [41]:
pdf_dir = "./dataset/pdf"
existing_files = os.listdir(image_dir)
for file in os.listdir(pdf_dir):
    pdf_file_path = os.path.join(pdf_dir, file)
    img_file_path = os.path.join(image_dir, file[:-3] + "png")
    file_extension = os.path.splitext(pdf_file_path)[-1]
    
    # create image
    try:
        if img_file_path not in existing_files and file_extension == ".pdf":
            page_lst = convert_from_path(pdf_file_path)
            img = page_lst[0]
            img.save(img_file_path)
    except Exception as e:
        print(f"Exception for pdf {pdf_file_path}: {e}")
        print("")

Exception for pdf ./dataset/pdf/.ipynb_checkpoints: Unable to get page count.
Syntax Warning: May not be a PDF file (continuing anyway)
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't find trailer dictionary
Syntax Error: Couldn't read xref table




In [42]:
print(len(os.listdir(image_dir)))

403


In [43]:
len(os.listdir(pdf_dir))

404

In [ ]:
# TODO: delete .ipynb_checkpoints
# !gsutil -m cp gs://gcs-public-data--labeled-patents/*.pdf  gs://labeled_patents/pdf

In [45]:
!gsutil -m cp ./dataset/images/*.png gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/images

Copying file://./dataset/images/computer_vision_10.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_11.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_12.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_1.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_13.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_19.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_15.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_14.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_20.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_2.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_5.png [Content-Type=image/png]...
Copying file://./dataset/images/computer_vision_3.png [Content-Type=image/png]...
Copying 

In [ ]:
# create csv with gcs_path (for png data) and with issuer information

In [47]:
query_string = """
#standardSQL
SELECT
    *
FROM
    `qwiklabs-gcp-00-373ac55d0e0a.labeled_patents_preprocessed.extracted_data`
"""

extracted_data_preprocessed = bq.query(query_string).to_dataframe()

In [49]:
image_classification_df = extracted_data_preprocessed.loc[:,["gcs_path","issuer"]]

In [53]:
def change_gcs_path_to_img_path(original):
    split_path = os.path.split(original)
    new_path = os.path.join("gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents/images" ,split_path[1])
    return new_path

def change_extension(original):
    return original[:-3] + "png"


In [52]:
image_classification_df["gcs_path"] = image_classification_df["gcs_path"].apply(change_gcs_path_to_img_path)

,gcs_path,issuer
0,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU
1,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU
2,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU
3,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU
4,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,EU
...,...,...
338,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,US
339,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,US
340,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,US
341,gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_pate...,US


In [54]:
image_classification_df["gcs_path"] = image_classification_df["gcs_path"].apply(change_extension)

In [60]:
image_classification_df.to_csv("./dataset/image_classification.csv", index=False, header=True)

In [61]:
!gsutil -m cp ./dataset/image_classification.csv gs://qwiklabs-gcp-00-373ac55d0e0a/labeled_patents

Copying file://./dataset/image_classification.csv [Content-Type=text/csv]...
/ [1/1 files][ 25.7 KiB/ 25.7 KiB] 100% Done                                    
Operation completed over 1 objects/25.7 KiB.                                     
